In [1]:
import copy
import pickle
import numpy as np
import networkx as nx
from itertools import product
from collections import OrderedDict

In [32]:
lambda_rate_sp = [2, 4, 6, 8, 10]
cost_diff_sp = [1.2, 1.4, 1.6, 1.8, 2.0]
func_num_sp = [10, 20, 30, 40, 50]
cpu_core_sp = [100, 200, 300, 400, 500]
cache_capacity_sp = [256, 512, 768, 1024]
soft_ddl_param_sp = [[0.1, 0.2], [0.2, 0.3], [0.3, 0.4], [0.4, 0.5]]

node_selection_strategy_sp = [
    "NS_min_dis_own",
    "NS_min_dis_all",
    "NS_min_user",
    "NS_max_node",
    "NS_exist_func",
]
path_selection_strategy_sp = ["PS_min_dis", "PS_max_cache", "PS_max_cpu"]
task_sorting_strategy_sp = [
    "TS_exec_time_asc",
    "TS_data_vol_asc",
    "TS_exec_time_to_data_vol_ratio_asc",
    "TS_closest_soft_ddl",
    "TS_highest_response_ratio",
]

combinations = list(
    product(lambda_rate_sp, cost_diff_sp, func_num_sp, cpu_core_sp,
            cache_capacity_sp, soft_ddl_param_sp, node_selection_strategy_sp,
            path_selection_strategy_sp, task_sorting_strategy_sp))

In [33]:
len(combinations)

750000

In [35]:
combinations[200]

(2,
 1.2,
 10,
 100,
 256,
 [0.3, 0.4],
 'NS_max_node',
 'PS_max_cache',
 'TS_exec_time_asc')

In [4]:
with open(r'outData/comm.pkl', 'wb') as file:
    pickle.dump(combinations, file)

In [10]:
# user.pkl
# 用户Id -> [
#            内网拓扑最近的边缘节点,
#            全网拓扑最近的边缘节点,
#            覆盖用户数最少的边缘节点,
#            邻居节点数最多的边缘节点,
#            以距离远近排序的边缘节点列表
#          ]

In [3]:
with open(r"outData/user-1.pkl", "rb") as file:
    user = pickle.load(file)
with open(r"outData/node.pkl", "rb") as file:
    node = pickle.load(file)

In [4]:
user[0]

array([[ 52,   1, 369],
       [ 93,   1, 146],
       [ 53,   1, 835],
       [ 12,   1, 222],
       [116,   1, 629],
       [ 50,   1, 549],
       [ 86,   1, 378],
       [ 22,   1, 902],
       [108,   6, 269],
       [  0,   6,  67],
       [ 92,   6,  63],
       [115,   6, 356],
       [ 21,   6, 430],
       [ 23,  24, 209],
       [ 87,  24, 147],
       [ 27,  24, 246],
       [ 34,  24, 167],
       [109,  24, 246]])

In [5]:
len(user)

816

In [6]:
##################################
# ---------节点选择策略---------- #
##################################
# 1.min_dis_own:选择距离最近的本服务提供商的基站
def NS_min_dis_own(userId, funcId=-1):
    serviceId = 1 if userId % 3 == 1 else (6 if userId % 3 == 2 else 24)
    sortedNdLst = sorted(user[userId], key=lambda x: x[2])
    for nd in sortedNdLst:
        if nd[1] == serviceId:
            return nd[0]

# 2.min_dis_all:选择距离最近的基站
def NS_min_dis_all(userId, funcId=-1):
    return min(user[userId], key=lambda x: x[2])[0]

# 3.min_user:选择覆盖用户数最少的基站
def NS_min_user(userId, funcId=-1):
    return min(user[userId], key=lambda x: node[x[0]][0])[0]

# 4.max_node:选择邻居基站数最多的基站
def NS_max_node(userId, funcId=-1):
    return max(user[userId], key=lambda x: node[x[0]][1])[0]

In [7]:
newUser = {}
for userId in user:
    newUser[userId] = [
        NS_min_dis_own(userId),
        NS_min_dis_all(userId),
        NS_min_user(userId),
        NS_max_node(userId),
        [x[0] for x in sorted(user[userId], key=lambda x: x[2])]
    ]

In [8]:
newUser[0]

[87,
 92,
 87,
 53,
 [92, 0, 93, 87, 34, 23, 12, 27, 109, 108, 115, 52, 86, 21, 50, 116, 53, 22]]

In [10]:
with open(r'outData/user-2.pkl', 'wb') as file:
    pickle.dump(newUser, file)

In [9]:
len(newUser)

816

In [11]:
with open(r"outData/graph-1.pkl", "rb") as file:
    graph = pickle.load(file)

In [12]:
graph

{'sub-1': <networkx.classes.digraph.DiGraph at 0x7febb811bc50>,
 'sub-6': <networkx.classes.digraph.DiGraph at 0x7febb8252a90>,
 'mmWave-24': <networkx.classes.digraph.DiGraph at 0x7febb86673d0>,
 'cross-net': <networkx.classes.digraph.DiGraph at 0x7febb82a5dd0>}

In [13]:
G_sub_1 = copy.deepcopy(graph["sub-1"])
G_sub_6 = copy.deepcopy(graph["sub-6"])
G_mmWave_24 = copy.deepcopy(graph["mmWave-24"])

In [14]:
for u, v, _ in G_sub_1.edges(data=True):
    G_sub_1[u][v]["weight"] = 1.0 / 1000.0

for u, v, _ in G_sub_6.edges(data=True):
    G_sub_6[u][v]["weight"] = 1.0 / 2000.0

for u, v, _ in G_mmWave_24.edges(data=True):
    G_mmWave_24[u][v]["weight"] = 1.0 / 4000.0

In [15]:
nx.shortest_path_length(G_sub_1, 30, weight="weight")

{30: 0,
 89: 0.001,
 52: 0.001,
 57: 0.001,
 93: 0.001,
 53: 0.001,
 1: 0.001,
 56: 0.001,
 12: 0.001,
 38: 0.001,
 76: 0.001,
 28: 0.001,
 22: 0.001,
 98: 0.001,
 84: 0.002,
 116: 0.002,
 50: 0.002,
 86: 0.002}

In [16]:
adjDict = {}
G_1 = {}
G_6 = {}
G_24 = {}
for nd in G_sub_1.nodes():
    G_1[nd] = dict(nx.shortest_path_length(G_sub_1, nd, weight="weight"))
for nd in G_sub_6.nodes():
    G_6[nd] = dict(nx.shortest_path_length(G_sub_6, nd, weight="weight"))
for nd in G_mmWave_24.nodes():
    G_24[nd] = dict(nx.shortest_path_length(G_mmWave_24, nd, weight="weight"))
adjDict[1] = G_1
adjDict[6] = G_6
adjDict[24] = G_24

In [17]:
adjDict[1]

{30: {30: 0,
  89: 0.001,
  52: 0.001,
  57: 0.001,
  93: 0.001,
  53: 0.001,
  1: 0.001,
  56: 0.001,
  12: 0.001,
  38: 0.001,
  76: 0.001,
  28: 0.001,
  22: 0.001,
  98: 0.001,
  84: 0.002,
  116: 0.002,
  50: 0.002,
  86: 0.002},
 89: {89: 0,
  30: 0.001,
  57: 0.001,
  53: 0.001,
  1: 0.001,
  56: 0.001,
  38: 0.001,
  76: 0.001,
  84: 0.001,
  28: 0.001,
  98: 0.001,
  52: 0.002,
  93: 0.002,
  12: 0.002,
  22: 0.002,
  116: 0.002,
  50: 0.002,
  86: 0.002},
 52: {52: 0,
  30: 0.001,
  57: 0.001,
  93: 0.001,
  53: 0.001,
  12: 0.001,
  116: 0.001,
  50: 0.001,
  86: 0.001,
  76: 0.001,
  28: 0.001,
  22: 0.001,
  89: 0.002,
  1: 0.002,
  56: 0.002,
  38: 0.002,
  98: 0.002,
  84: 0.002},
 57: {57: 0,
  30: 0.001,
  89: 0.001,
  52: 0.001,
  53: 0.001,
  1: 0.001,
  56: 0.001,
  38: 0.001,
  76: 0.001,
  84: 0.001,
  28: 0.001,
  22: 0.001,
  98: 0.001,
  93: 0.002,
  12: 0.002,
  116: 0.002,
  50: 0.002,
  86: 0.002},
 93: {93: 0,
  30: 0.001,
  52: 0.001,
  53: 0.001,
  12: 0.

In [18]:
sub_1 = np.array(
    [30, 89, 52, 57, 93, 53, 1, 56, 12, 116, 38, 50, 86, 76, 84, 28, 22, 98])
sub_6 = np.array([
    103, 54, 108, 45, 95, 36, 58, 14, 6, 11, 0, 96, 88, 25, 104, 113, 80, 81,
    94, 7, 31, 92, 115, 39, 21, 72, 4, 9, 105, 85, 37, 63, 107, 17, 79, 26
])
mmWave_24 = np.array([
    32, 55, 124, 118, 101, 3, 106, 97, 66, 19, 68, 13, 122, 70, 23, 59, 15, 47,
    10, 87, 102, 65, 29, 2, 73, 5, 75, 114, 117, 77, 33, 35, 60, 8, 46, 111,
    112, 71, 43, 61, 27, 51, 69, 121, 90, 16, 41, 64, 67, 34, 99, 120, 123, 62,
    100, 74, 109, 42, 119, 48, 78, 24, 83, 110, 20, 44, 91, 40, 82, 18, 49
])
cost_diff = 1.2
G_cross_net = copy.deepcopy(graph["cross-net"])
for u, v, _ in G_cross_net.edges(data=True):
    if u in sub_1:
        bandwidth = 1000.0
    elif u in sub_6:
        bandwidth = 2000.0
    else:
        bandwidth = 4000.0
    if ((u in sub_1 and v in sub_1) or (u in sub_6 and v in sub_6)
            or (u in mmWave_24 and v in mmWave_24)):
        G_cross_net[u][v]["weight"] = 1.0 / bandwidth
    else:
        G_cross_net[u][v]["weight"] = cost_diff / bandwidth
G_tmp = {}
for nd in G_cross_net.nodes():
    G_tmp[nd] = dict(nx.shortest_path_length(G_cross_net, nd, weight="weight"))
adjDict[cost_diff] = G_tmp

In [19]:
cost_diff = 1.4
G_cross_net = copy.deepcopy(graph["cross-net"])
for u, v, _ in G_cross_net.edges(data=True):
    if u in sub_1:
        bandwidth = 1000.0
    elif u in sub_6:
        bandwidth = 2000.0
    else:
        bandwidth = 4000.0
    if ((u in sub_1 and v in sub_1) or (u in sub_6 and v in sub_6)
            or (u in mmWave_24 and v in mmWave_24)):
        G_cross_net[u][v]["weight"] = 1.0 / bandwidth
    else:
        G_cross_net[u][v]["weight"] = cost_diff / bandwidth
G_tmp = {}
for nd in G_cross_net.nodes():
    G_tmp[nd] = dict(nx.shortest_path_length(G_cross_net, nd, weight="weight"))
adjDict[cost_diff] = G_tmp

In [20]:
cost_diff = 1.6
G_cross_net = copy.deepcopy(graph["cross-net"])
for u, v, _ in G_cross_net.edges(data=True):
    if u in sub_1:
        bandwidth = 1000.0
    elif u in sub_6:
        bandwidth = 2000.0
    else:
        bandwidth = 4000.0
    if ((u in sub_1 and v in sub_1) or (u in sub_6 and v in sub_6)
            or (u in mmWave_24 and v in mmWave_24)):
        G_cross_net[u][v]["weight"] = 1.0 / bandwidth
    else:
        G_cross_net[u][v]["weight"] = cost_diff / bandwidth
G_tmp = {}
for nd in G_cross_net.nodes():
    G_tmp[nd] = dict(nx.shortest_path_length(G_cross_net, nd, weight="weight"))
adjDict[cost_diff] = G_tmp

In [21]:
cost_diff = 1.8
G_cross_net = copy.deepcopy(graph["cross-net"])
for u, v, _ in G_cross_net.edges(data=True):
    if u in sub_1:
        bandwidth = 1000.0
    elif u in sub_6:
        bandwidth = 2000.0
    else:
        bandwidth = 4000.0
    if ((u in sub_1 and v in sub_1) or (u in sub_6 and v in sub_6)
            or (u in mmWave_24 and v in mmWave_24)):
        G_cross_net[u][v]["weight"] = 1.0 / bandwidth
    else:
        G_cross_net[u][v]["weight"] = cost_diff / bandwidth
G_tmp = {}
for nd in G_cross_net.nodes():
    G_tmp[nd] = dict(nx.shortest_path_length(G_cross_net, nd, weight="weight"))
adjDict[cost_diff] = G_tmp

In [22]:
cost_diff = 2.0
G_cross_net = copy.deepcopy(graph["cross-net"])
for u, v, _ in G_cross_net.edges(data=True):
    if u in sub_1:
        bandwidth = 1000.0
    elif u in sub_6:
        bandwidth = 2000.0
    else:
        bandwidth = 4000.0
    if ((u in sub_1 and v in sub_1) or (u in sub_6 and v in sub_6)
            or (u in mmWave_24 and v in mmWave_24)):
        G_cross_net[u][v]["weight"] = 1.0 / bandwidth
    else:
        G_cross_net[u][v]["weight"] = cost_diff / bandwidth
G_tmp = {}
for nd in G_cross_net.nodes():
    G_tmp[nd] = dict(nx.shortest_path_length(G_cross_net, nd, weight="weight"))
adjDict[cost_diff] = G_tmp

In [24]:
sorted(adjDict[1][30].items(), key=lambda x: x[1])

[(30, 0),
 (89, 0.001),
 (52, 0.001),
 (57, 0.001),
 (93, 0.001),
 (53, 0.001),
 (1, 0.001),
 (56, 0.001),
 (12, 0.001),
 (38, 0.001),
 (76, 0.001),
 (28, 0.001),
 (22, 0.001),
 (98, 0.001),
 (84, 0.002),
 (116, 0.002),
 (50, 0.002),
 (86, 0.002)]

In [25]:
OrderedDict(sorted(adjDict[1][30].items(), key=lambda x: x[1]))

OrderedDict([(30, 0),
             (89, 0.001),
             (52, 0.001),
             (57, 0.001),
             (93, 0.001),
             (53, 0.001),
             (1, 0.001),
             (56, 0.001),
             (12, 0.001),
             (38, 0.001),
             (76, 0.001),
             (28, 0.001),
             (22, 0.001),
             (98, 0.001),
             (84, 0.002),
             (116, 0.002),
             (50, 0.002),
             (86, 0.002)])

In [26]:
for key in OrderedDict(sorted(adjDict[1][30].items(), key=lambda x: x[1])):
    print(key)

30
89
52
57
93
53
1
56
12
38
76
28
22
98
84
116
50
86


In [27]:
for k1 in adjDict:
    for k2 in adjDict[k1]:
        adjDict[k1][k2] = OrderedDict(sorted(adjDict[k1][k2].items(), key=lambda x: x[1]))

In [28]:
for key in adjDict:
    print(key, len(adjDict[key]))

1 18
6 36
24 71
1.2 125
1.4 125
1.6 125
1.8 125
2.0 125


In [29]:
with open(r'outData/graph-2.pkl', 'wb') as file:
    pickle.dump(adjDict, file)

In [30]:
type(adjDict[1.2][30])

collections.OrderedDict

In [31]:
adjDict[1.2][30][30]

0

In [5]:
import pandas as pd

res = pd.read_csv("res/3.csv", sep='|')
res.to_csv("res/3.csv", index=False)

In [8]:
item = 1
start = 0
end = 6400
for i in range(40):
    print("item =", item)
    print("start =", start)
    print("end =", end)
    item += 1
    start = end
    end += 6400

item = 1
start = 0
end = 6400
item = 2
start = 6400
end = 12800
item = 3
start = 12800
end = 19200
item = 4
start = 19200
end = 25600
item = 5
start = 25600
end = 32000
item = 6
start = 32000
end = 38400
item = 7
start = 38400
end = 44800
item = 8
start = 44800
end = 51200
item = 9
start = 51200
end = 57600
item = 10
start = 57600
end = 64000
item = 11
start = 64000
end = 70400
item = 12
start = 70400
end = 76800
item = 13
start = 76800
end = 83200
item = 14
start = 83200
end = 89600
item = 15
start = 89600
end = 96000
item = 16
start = 96000
end = 102400
item = 17
start = 102400
end = 108800
item = 18
start = 108800
end = 115200
item = 19
start = 115200
end = 121600
item = 20
start = 121600
end = 128000
item = 21
start = 128000
end = 134400
item = 22
start = 134400
end = 140800
item = 23
start = 140800
end = 147200
item = 24
start = 147200
end = 153600
item = 25
start = 153600
end = 160000
item = 26
start = 160000
end = 166400
item = 27
start = 166400
end = 172800
item = 28
start = 17